In [1]:
import os

curr_dir = os.path.split(os.getcwd())[1]
if curr_dir != "irl-environment-design":
    os.chdir("..")

import numpy as np

from src.utils.make_environment import (
    transition_matrix,
    Environment,
    insert_walls_into_T,
)

from src.utils.constants import GenParamTuple# candidate_environments_args["n_environments"] = 50


np.set_printoptions(linewidth=160, precision=2)

from src.utils.environment_design import EnvironmentDesign

## Make true environment

In [2]:
# 0. Setup
## 0.2 Setup the environment
N, M = 7,7

# TODO describe world
agent_R = np.zeros((N, M))
agent_R[N-1, 0] = 1
agent_R[N-1, M-1] = 3



# Start by making the agent we want to learn the policy of
agent_gamma = 0.6
p_true=0.7

agent_R = agent_R.flatten()
goal_states = np.where(agent_R != 0)[0]


wall_states = [14] #TODO: why do we need this

T_true = transition_matrix(N, M, p=p_true, absorbing_states=goal_states)
T_True = insert_walls_into_T(T=T_true, wall_indices=wall_states)


def custom_transition_func(p):

    _T = transition_matrix(N=7, M=7, p=p, absorbing_states=goal_states)
    _T = insert_walls_into_T(T=_T, wall_indices=wall_states)
    return _T

def custom_gamma_func(gamma):
    return gamma

def custom_reward_func(*reward_func):
    return reward_func

#Create parameter ranges
resolution = 15
p_range = np.linspace(0.7, 0.9, resolution)
gamma_range = np.linspace(0.6, 0.9, resolution)


gamma_range = gamma_range.reshape(1, resolution)
p_range = p_range.reshape(1, resolution)

true_params = GenParamTuple(T = custom_transition_func(p_true), gamma=agent_gamma, R=agent_R)




big_small = Environment(
    N=N,
    M=M,
    reward_function = custom_reward_func,
    transition_function=custom_transition_func,
    gamma = custom_gamma_func,
    wall_states=wall_states,
    start_state=0,
    goal_states=goal_states
)

In [3]:
env_design = EnvironmentDesign(base_environment=big_small, 
                               user_params=true_params, 
                               learn_what = ["gamma", "T"],
                               parameter_ranges_R=None,
                               parameter_ranges_gamma=gamma_range,
                               parameter_ranges_T=p_range)

Generated parameter mesh of shape:  (15, 15)


In [4]:
candidate_environments_args = {}
candidate_environments_args["generate_how"] = "entropy_BM"

env_design.run_n_episodes(n_episodes = 15,
                          candidate_environments_args=candidate_environments_args,
                          verbose=True)

Started episode 0.
Finished episode 0.
Started episode 1.
Beginning calculation of log-likelihood. Calculating 225 samples.
Mean Parameters: [0.6863480080274913, 0.7910756448235351]
Computed Region of Interest. Size = 0.48
Finished BM Search. Entropy: 1.081770401476498.
Finished episode 1.
Started episode 2.
Beginning calculation of log-likelihood. Calculating 225 samples.
Mean Parameters: [0.663002573362093, 0.7830207654894656]
Computed Region of Interest. Size = 0.35
Finished BM Search. Entropy: 1.0154436148434798.
Finished episode 2.
Started episode 3.
Beginning calculation of log-likelihood. Calculating 225 samples.
Mean Parameters: [0.6512693865309769, 0.7711833934343407]
Computed Region of Interest. Size = 0.26
Finished BM Search. Entropy: 0.9229936905441962.
Finished episode 3.
Started episode 4.
Beginning calculation of log-likelihood. Calculating 225 samples.
Mean Parameters: [0.6449203530801908, 0.7586531432643634]
Computed Region of Interest. Size = 0.2
Finished BM Search. E

In [5]:
'''
user parameters consist of
- parametrizations of reward, transition and gamma that user supplies
- ranges for the unknown parameters
- resolution of the grid search
'''

'\nuser parameters consist of\n- parametrizations of reward, transition and gamma that user supplies\n- ranges for the unknown parameters\n- resolution of the grid search\n'